Will act as a live document as I test and play around with analyzing steam reviews via a valve sourced dataset

First we need to get our dataset


In [11]:
import pandas as pd

df = pd.read_json("https://store.steampowered.com/appreviews/1533420?json=1")

df.info()

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [50]:
import requests

URL= "https://store.steampowered.com/appreviews/1533420?json=1"
cursor = '*'
PARAMS = {
    'language':'english',
    'cursor': cursor,
    'num_per_page' : 100,
}


r = requests.get(URL, PARAMS)
r=r.json()

In [51]:
print(r)

print(r["query_summary"]['total_reviews'])
print(r["cursor"])

{'success': 1, 'query_summary': {'num_reviews': 100, 'review_score': 9, 'review_score_desc': 'Overwhelmingly Positive', 'total_positive': 1247, 'total_negative': 18, 'total_reviews': 1265}, 'reviews': [{'recommendationid': '117153662', 'author': {'steamid': '76561198137122684', 'num_games_owned': 269, 'num_reviews': 1, 'playtime_forever': 1600, 'playtime_last_two_weeks': 1600, 'playtime_at_review': 784, 'last_played': 1655822267}, 'language': 'english', 'review': 'this scratches my adhd riddled brain more than anything i have ever played\n\nmaking a decision every 3-4 seconds in the maybe the fastest puzzle game ever made, accompanied by breakcore 10 hours (cure for adhd) soundtrack by machine girl, this stuff is just juice for the flesh dome', 'timestamp_created': 1655511937, 'timestamp_updated': 1655511937, 'voted_up': True, 'votes_up': 331, 'votes_funny': 12, 'weighted_vote_score': '0.937777340412139893', 'comment_count': 9, 'steam_purchase': True, 'received_for_free': False, 'writt